In [77]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import warnings
import rasterio as rio

%matplotlib inline

from netCDF4 import Dataset
from pyhdf.SD import SD, SDC
from skimage.transform import resize

In [78]:
ds = Dataset('C:\\Users\\Elle\\Downloads\\ndvi3g_geo_v1_1985_0712.nc4')

In [108]:
from collections import Counter

from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.metrics import  confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from skimage.transform import resize

In [80]:
lidar_dem_path = 'C:\\Users\\Elle\\Documents\\w210\\lower_scaled_gfsad.tif'
with rio.open(lidar_dem_path) as lidar_dem:
    im_array = lidar_dem.read()
    lidar_dem.bounds

In [81]:
for i in range(10):
    print(i, np.count_nonzero(im_array == i), round(np.count_nonzero(im_array == i)/(4320*2160)*100,2))

0 6241070 66.88
1 27721 0.3
2 16840 0.18
3 33168 0.36
4 118782 1.27
5 117480 1.26
6 54815 0.59
7 125240 1.34
8 208092 2.23
9 2387992 25.59


In [82]:
# Get a list of cropland and their classes
im_array = im_array.reshape((2160,4320))

def apply_mask(pixel):
    if pixel == 9:
        return 0
    else:
        return pixel

filter_function = np.vectorize(apply_mask)
unmasked_pixels = filter_function(im_array)

land_pixels = np.nonzero(unmasked_pixels) 
# print(np.unique(imarray))
land_pixel_classes = im_array[land_pixels].tolist()
print(len(land_pixel_classes))

702138


In [83]:
land_indices = land_pixels 
non_zero_indices = np.array(land_indices)
clean_frame = non_zero_indices.T
print(clean_frame.shape)
n = clean_frame.shape[0]
non_zeros = np.nonzero(im_array)

clean_frame_2 = clean_frame 
clean_frame_df = pd.DataFrame({'lon': clean_frame[:,0], 'lat': clean_frame[:,1]})
clean_frame_df['labels'] = land_pixel_classes

(702138, 2)


In [28]:
clean_frame_df

,lon,lat,labels
0,294,2469,8
1,295,2467,8
2,295,2468,8
3,300,2466,8
4,300,2467,8
...,...,...,...
702133,1745,1328,8
702134,1745,1334,8
702135,1745,1339,8
702136,1745,1340,8


In [91]:
tifs = ['1985', '1990', '1995', '2000', '2005']
years = ['1985', '1990', '1995', '2000', '2005', '2010', '2015']
lidar_dem_path = 'C:\\Users\\Elle\\Documents\\w210\\1985.tif'
times_series_labels = np.zeros((5, 2160, 4320))
for i in range(len(tifs)):
    lidar_dem_path = 'C:\\Users\\Elle\\Documents\\w210\\' + tifs[i] +'.tif'
    with rio.open(lidar_dem_path) as lidar_dem:
        array = lidar_dem.read() 
        array = array.reshape(2160, 4320)
        clean_frame_df[str(tifs[i])] = array[land_indices].reshape(n,1)

times_series_labels[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
clean_frame_df[clean_frame_df['1985']>2000].groupby(['labels']).count()

,lon,lat,1985,1990,1995,2000,2005
labels,,,,,,,
1,10944,10944,10944,10944,10944,10944,10944
2,6774,6774,6774,6774,6774,6774,6774
3,8528,8528,8528,8528,8528,8528,8528
4,2280,2280,2280,2280,2280,2280,2280
5,1718,1718,1718,1718,1718,1718,1718
6,495,495,495,495,495,495,495
7,4680,4680,4680,4680,4680,4680,4680
8,1456,1456,1456,1456,1456,1456,1456


In [92]:
df = clean_frame_df[['labels','1985', '1990', '1995', '2000', '2005']]

In [93]:
df[df['labels']>3].quantile(.75)

labels     8.000
1985      65.040
1990      73.682
1995      78.392
2000      79.313
2005      83.540
Name: 0.75, dtype: float64

In [33]:
clean_frame_df[df['labels']<4].quantile(.25)

lon        598.000
lat       2701.000
labels       1.000
1985        50.897
1990        81.681
1995       107.460
2000       125.600
2005       133.300
Name: 0.25, dtype: float64

In [94]:
df_85, df_90, df_95, df_00, df_05, df_10, df_15  =  pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()


df_lists = [df_85, df_90, df_95, df_00, df_05, df_10, df_15]

for i in range(len(df_lists)):
    df_lists[i]['lat'] = clean_frame_df['lat']
    df_lists[i]['lon'] = clean_frame_df['lon']
    if i < 5:
        df_lists[i]['labels'] = clean_frame_df[tifs[i]]
    

In [95]:
for i in range(len(df_lists[:5])):
    df_lists[i]['year'] = years[i]

In [96]:
measures = ['max_y1', 'min_y1', 'mean_y1', 'var_y1', 'max_y2', 'min_y2', 'mean_y2', 'var_y2']
ndvi_list = os.listdir('C:\\Users\\Elle\\Documents\\w210\\data\\ndvi')
def retrieve_ndvi(indices, length, year):
            path = 'C:\\Users\\Elle\\Documents\\w210\\data\\ndvi\\ndvi3g_geo_v1_'
            file_1h = path + year + '_0106.nc4'
            file_2h = path + year + '_0712.nc4' 
            ds_1, ds_2 = np.array(Dataset(file_1h)['ndvi']) , np.array(Dataset(file_2h)['ndvi'])
            max_y1 = np.max(ds_1, axis = 0)[indices].reshape(length,1)
            min_y1 = np.min(ds_1, axis = 0)[indices].reshape(length,1)
            var_y1 = np.var(ds_1, axis = 0)[indices].reshape(length,1)
            mean_y1= np.mean(ds_1, axis = 0)[indices].reshape(length,1)
            max_y2 = np.max(ds_2, axis = 0)[indices].reshape(length,1)
            min_y2 = np.min(ds_2, axis = 0)[indices].reshape(length,1)
            var_y2 = np.var(ds_2, axis = 0)[indices].reshape(length,1)
            mean_y2 = np.mean(ds_2, axis = 0)[indices].reshape(length,1)
            return max_y1, min_y1, mean_y1, var_y1, max_y2, min_y2, mean_y2, var_y2

for i in range(len(df_lists)):
    df_lists[i]['max_y1_ndvi'], df_lists[i]['min_y1_ndvi'], df_lists[i]['mean_y1_ndvi'], df_lists[i]['var_y1_ndvi'], df_lists[i]['max_y2_ndvi'], df_lists[i]['min_y2_ndvi'], df_lists[i]['mean_y2_ndvi'], df_lists[i]['var_y2_ndvi'] = retrieve_ndvi(land_indices, n, years[i])


C:\Users\Elle\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  import sys


In [37]:
print(n)

702138


In [100]:
measures = ['aet', 'def', 'PDSI', 'pet', 'ppt', 'q', 'soil', 'srad', 'tmax', 'tmin', 'vap', 'vpd', 'ws'] 

def extract_nc(indices, length, year, variable):
    path = 'C:\\Users\\Elle\\Documents\\w210\\data\\climate\\' + year + '\\'
    full_path = path + 'TerraClimate_' + variable +'_' + year + '.nc'
    ds = np.array(Dataset(full_path)[variable])
    max_y1 = np.max(ds, axis = 0)
    min_y1 = np.min(ds, axis = 0)
    var_y1 = np.var(ds, axis = 0)
    mean_y1 = np.mean(ds, axis = 0)
    max_y1 = resize(max_y1, (2160, 4320))[indices].reshape(length,1)
    min_y1 = resize(min_y1, (2160, 4320))[indices].reshape(length,1)
    var_y1 = resize(var_y1, (2160, 4320))[indices].reshape(length,1)
    mean_y1 = resize(mean_y1, (2160, 4320))[indices].reshape(length,1)
    return max_y1, min_y1, var_y1, mean_y1


for i in measures:
    for j in range(len(df_lists)):
        df_lists[j][i+'_max'], df_lists[j][i+'_min'], df_lists[j][i+'_var'], df_lists[j][i+'_mean'] = extract_nc(land_indices, n, years[j], i)
        print(i+","+str(j))

aet,0
aet,1
aet,2
aet,3
aet,4
aet,5
aet,6
def,0
def,1
def,2
def,3
def,4
def,5
def,6
PDSI,0
PDSI,1
PDSI,2
PDSI,3
PDSI,4
PDSI,5
PDSI,6
pet,0
pet,1
pet,2
pet,3
pet,4
pet,5
pet,6
ppt,0
ppt,1
ppt,2
ppt,3
ppt,4
ppt,5
ppt,6
q,0
q,1
q,2
q,3
q,4
q,5
q,6
soil,0
soil,1
soil,2
soil,3
soil,4
soil,5
soil,6
srad,0
srad,1
srad,2
srad,3
srad,4
srad,5
srad,6
tmax,0
tmax,1
tmax,2
tmax,3
tmax,4
tmax,5
tmax,6
tmin,0
tmin,1
tmin,2
tmin,3
tmin,4
tmin,5
tmin,6
vap,0
vap,1
vap,2
vap,3
vap,4
vap,5
vap,6
vpd,0
vpd,1
vpd,2
vpd,3
vpd,4
vpd,5
vpd,6
ws,0
ws,1
ws,2
ws,3
ws,4
ws,5
ws,6


In [101]:
measures_2 = ['aet', 'def', 'pet', 'ppt', 'q', 'soil', 'srad', 'tmax', 'tmin', 'vap', 'vpd', 'ws'] 

def extract_nc_lt(indices, length, variable):
    path = 'C:\\Users\\Elle\\Documents\\w210\\data\\climate\\lt\\'
    full_path = path + 'TerraClimate19812010_' + variable  + '.nc'
    print(full_path)
    ds = np.array(Dataset(full_path)[variable])
    max_y1 = np.max(ds, axis = 0)
    min_y1 = np.min(ds, axis = 0)
    var_y1 = np.var(ds, axis = 0)
    mean_y1= np.mean(ds, axis = 0)
    max_y1 = resize(max_y1, (2160, 4320))[indices].reshape(length,1)
    min_y1 = resize(min_y1, (2160, 4320))[indices].reshape(length,1)
    var_y1 = resize(var_y1, (2160, 4320))[indices].reshape(length,1)
    mean_y1 = resize(mean_y1, (2160, 4320))[indices].reshape(length,1)
    return max_y1, min_y1, var_y1, mean_y1

for i in measures_2:
    clean_frame_df[i+'_lt_max'], clean_frame_df[i+'_lt_min'], clean_frame_df[i+'_lt_var'], clean_frame_df[i+'_lt_mean'] = extract_nc_lt(land_indices, n, i)

C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_aet.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_def.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_pet.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_ppt.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_q.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_soil.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_srad.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_tmax.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_tmin.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_vap.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_vpd.nc
C:\Users\Elle\Documents\w210\data\climate\lt\TerraClimate19812010_ws.nc


In [103]:
for j in df_lists:
    for i in measures_2:
        j[i+'_lt_max'] = clean_frame_df[i+'_lt_max'] 
        j[i+'_lt_min'] = clean_frame_df[i+'_lt_min'] 
        j[i+'_lt_var'] = clean_frame_df[i+'_lt_var']
        j[i+'_lt_mean'] = clean_frame_df[i+'_lt_mean']

In [104]:
combined_df = pd.concat([df_85, df_90, df_95, df_00, df_05])
len(combined_df.columns)

112

In [45]:
combined_df.columns

Index(['lat', 'lon', 'labels', 'year', 'max_y1_ndvi', 'min_y1_ndvi',
       'mean_y1_ndvi', 'var_y1_ndvi', 'max_y2_ndvi', 'min_y2_ndvi',
       ...
       'vap_lt_var', 'vap_lt_mean', 'vpd_lt_max', 'vpd_lt_min', 'vpd_lt_var',
       'vpd_lt_mean', 'ws_lt_max', 'ws_lt_min', 'ws_lt_var', 'ws_lt_mean'],
      dtype='object', length=112)

In [105]:
features = ['max_y1_ndvi', 'min_y1_ndvi', 'mean_y1_ndvi', 'var_y1_ndvi', 'max_y2_ndvi', 'min_y2_ndvi', 'mean_y2_ndvi', 'var_y2_ndvi', 'aet_max', 'aet_min', 'aet_var', 'aet_mean', 'def_max', 'def_min', 'def_var', 'def_mean', 'PDSI_max', 'PDSI_min', 'PDSI_var', 'PDSI_mean', 'pet_max', 'pet_min', 'pet_var', 'pet_mean', 'ppt_max', 'ppt_min', 'ppt_var', 'ppt_mean', 'q_max', 'q_min', 'q_var', 'q_mean', 'soil_max', 'soil_min', 'soil_var', 'soil_mean', 'srad_max', 'srad_min', 'srad_var', 'srad_mean', 'tmax_max', 'tmax_min', 'tmax_var', 'tmax_mean', 'tmin_max', 'tmin_min', 'tmin_var', 'tmin_mean', 'vap_max', 'vap_min', 'vap_var', 'vap_mean', 'vpd_max', 'vpd_min', 'vpd_var', 'vpd_mean', 'ws_max', 'ws_min', 'ws_var', 'ws_mean', 'aet_lt_max', 'aet_lt_min', 'aet_lt_var', 'aet_lt_mean', 'def_lt_max', 'def_lt_min', 'def_lt_var', 'def_lt_mean', 'pet_lt_max', 'pet_lt_min', 'pet_lt_var', 'pet_lt_mean', 'ppt_lt_max', 'ppt_lt_min', 'ppt_lt_var', 'ppt_lt_mean', 'q_lt_max', 'q_lt_min', 'q_lt_var', 'q_lt_mean', 'soil_lt_max', 'soil_lt_min', 'soil_lt_var', 'soil_lt_mean', 'srad_lt_max', 'srad_lt_min', 'srad_lt_var', 'srad_lt_mean', 'tmax_lt_max', 'tmax_lt_min', 'tmax_lt_var', 'tmax_lt_mean', 'tmin_lt_max', 'tmin_lt_min', 'tmin_lt_var', 'tmin_lt_mean', 'vap_lt_max', 'vap_lt_min', 'vap_lt_var', 'vap_lt_mean', 'vpd_lt_max', 'vpd_lt_min', 'vpd_lt_var', 'vpd_lt_mean', 'ws_lt_max', 'ws_lt_min', 'ws_lt_var', 'ws_lt_mean']

In [49]:
combined_df['irrigated'] = [1 if x > 100 else 0 for x in combined_df['labels']]

In [ ]:
combined_df[combined_df['irrigated'] == 1].shape

In [109]:
data = combined_df[features]
labels = combined_df['labels']
ncores = os.cpu_count()
train_data_unb, test_data_unb, train_labels_unb, test_labels_unb = train_test_split(data, labels, test_size=0.3)
clf_unb = RandomForestRegressor(n_estimators=40 , min_samples_leaf=2, max_depth=50, n_jobs=ncores)


In [ ]:
%time clf_unb.fit(train_data_unb, train_labels_unb)

In [50]:
data = combined_df[features]
labels = combined_df['irrigated']
ncores = os.cpu_count()
train_data_unb, test_data_unb, train_labels_unb, test_labels_unb = train_test_split(data, labels, test_size=0.3)
clf_unb = RandomForestClassifier(n_estimators=40 , min_samples_leaf=2, max_depth=50, n_jobs=ncores)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)


Wall time: 10min 53s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=50, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=8,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [51]:
%time predictions_unb = clf_unb.predict(test_data_unb)
print("acc",accuracy_score(test_labels_unb, predictions_unb))
print("kappa", cohen_kappa_score(test_labels_unb, predictions_unb))
print(classification_report(test_labels_unb, predictions_unb))

Wall time: 9.01 s
acc 0.9452282409820671
kappa 0.8630000570858083
              precision    recall  f1-score   support

           0       0.95      0.97      0.96    755689
           1       0.92      0.88      0.90    297518

    accuracy                           0.95   1053207
   macro avg       0.94      0.93      0.93   1053207
weighted avg       0.94      0.95      0.94   1053207



In [ ]:
combined_df.columns

In [52]:
test, train = combined_df[combined_df['year'] == '2005'], combined_df[combined_df['year'] != '2005']
train_irr = train[train['irrigated'] == 1]
train_non = train[train['irrigated'] == 0]
train_non = train_non.sample(frac=0.5, replace=False, random_state=1)
train = pd.concat([train_irr, train_non])
test_labels, train_labels = test['irrigated'], train['irrigated']
test_data, train_data = test[features], train[features]

In [53]:
clf_unb = ExtraTreesClassifier(n_estimators=200, min_samples_leaf=2, max_depth=50, n_jobs=ncores)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
%time clf_unb.fit(train_data, train_labels)

Wall time: 9min 52s


ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=50, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=2, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=8,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [54]:
%time predictions= clf_unb.predict(test_data)
print("acc",accuracy_score(test_labels, predictions))
print("kappa", cohen_kappa_score(test_labels, predictions))
print(classification_report(test_labels, predictions))

Wall time: 11 s
acc 0.9153841552515317
kappa 0.79504236774018
              precision    recall  f1-score   support

           0       0.94      0.94      0.94    496369
           1       0.86      0.85      0.85    205769

    accuracy                           0.92    702138
   macro avg       0.90      0.90      0.90    702138
weighted avg       0.92      0.92      0.92    702138



In [57]:
clean_frame_df['predictions'] = predictions

In [72]:
clean_frame_df['2005_binary'] = [1 if x > 100 else 0 for x in clean_frame_df['2005']]
clean_frame_df['2000_binary'] = [1 if x > 100 else 0 for x in clean_frame_df['2000']]
clean_frame_df['1995_binary'] = [1 if x > 100 else 0 for x in clean_frame_df['1995']]
clean_frame_df['1990_binary'] = [1 if x > 100 else 0 for x in clean_frame_df['1990']]
clean_frame_df['1985_binary'] = [1 if x > 100 else 0 for x in clean_frame_df['1985']]

In [66]:
clean_frame_df['predictions'][clean_frame_df['predictions']==1].count() 

203213

In [ ]:
clean_frame_df['predictions'][clean_frame_df['predictions']==1].count() 

In [64]:
clean_frame_df['predictions'][clean_frame_df['predictions']==1].count() 

203213

In [62]:
clean_frame_df['2005_binary'][clean_frame_df['2005_binary']==1].count() 

205769

In [67]:
clean_frame_df['2000_binary'][clean_frame_df['2000_binary']==1].count() 

203465

In [69]:
clean_frame_df['2005_binary'][clean_frame_df['2005_binary']==1].count() 

199211

In [ ]:
clean_frame_df['2005_binary'][clean_frame_df['2005_binary']==1].count() 

In [71]:
clean_frame_df['1990_binary'][clean_frame_df['1990_binary']==1].count() 

195514

In [73]:
clean_frame_df['1985_binary'][clean_frame_df['1985_binary']==1].count() 

186986

In [ ]:
clean_frame_df['1990_binary'][clean_frame_df['1990_binary']==1].count() 

In [ ]:
clean_frame_df['2000_binary'][clean_frame_df['2000_binary']==1].count() 

In [55]:
df_import = pd.DataFrame()
df_import['feature'] = features
df_import['importances'] = clf_unb.feature_importances_
df_import=df_import.sort_values('importances', ascending=False)
df_import.head(n = 10)

,feature,importances
98,vap_lt_var,0.039653
90,tmax_lt_var,0.030931
86,srad_lt_var,0.021738
70,pet_lt_var,0.018637
46,tmin_var,0.017236
50,vap_var,0.016958
88,tmax_lt_max,0.016044
94,tmin_lt_var,0.015989
66,def_lt_var,0.014926
42,tmax_var,0.014882


In [ ]:
variable = 'aet'
year = '1985'
path = 'C:\\Users\\Elle\\Documents\\w210\\data\\climate\\' + year + '\\'
full_path = path + 'TerraClimate_' + variable +'_' + year + '.nc'
print(full_path)
ds = np.array(Dataset(full_path)[variable])
max_y1 = np.max(ds, axis = 0)
min_y1 = np.min(ds, axis = 0)
var_y1 = np.var(ds, axis = 0)
mean_y1= np.mean(ds, axis = 0)
print(mean_y1.shape)
mean = resize(mean_y1, (2160, 4320))
print(mean.shape)
df_85['mean_aet'] = mean[land_indices]

# data = resize(ds, (2160, 4320))

In [ ]:
df_95

In [ ]:
print(lidar_dem)

In [ ]:
test = np.array(ds['ndvi'])

In [ ]:
np.count_nonzero((test < 1) & (test > 0))

In [ ]:
ds = Dataset('C:\\Users\\Elle\\Documents\\w210\\data\\ndvi\\ndvi3g_geo_v1_1985_0106.nc4')

In [ ]:
test = np.array(ds['ndvi'])

In [ ]:
test 